In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType, DateType
from pyspark.sql.functions import *

#set up spark (session)
sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("BatchPipeline")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")

spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

#set up hadoop fs configuration
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

#retrieve data from bucket
#google storage file path
#make sure this is a seperate bucket that only has the correct files in there
gsc_file_path = 'gs://de_jads_batch_data/' # bucket name !

dataSchema = StructType(
    [StructField("home_team", StringType(), True),
    StructField("away_team", StringType(), True),
    StructField("home_score", LongType(), True),
    StructField("home_xg", DoubleType(), True),
    StructField("home_penalty", LongType(), True),
    StructField("away_score", LongType(), True),
    StructField("away_xg", DoubleType(), True),
    StructField("away_penalty", LongType(), True),
    StructField("home_manager", StringType(), True),
    StructField("home_captain", StringType(), True),
    StructField("away_manager", StringType(), True),
    StructField("away_captain", StringType(), True),
    StructField("home_goals", StringType(), True),
    StructField("away_goals", StringType(), True),
    StructField("Attendance", LongType(), True),
    StructField("Venue", StringType(), True),
    StructField("Officials", StringType(), True),
    StructField("Date", StringType(), True),
    StructField("Score", StringType(), True),
    StructField("Referee", StringType(), True),
    StructField("Notes", StringType(), True),
    StructField("Round", StringType(), True),
    StructField("Host", StringType(), True),
    StructField("Year", LongType(), True)
    ])

#add all decades to one dataframe
matches = spark.read.format("csv").schema(dataSchema).option("header", "true") \
    .load(gsc_file_path+'*.csv')

matches.printSchema()



root
 |-- home_team: string (nullable = true)
 |-- away_team: string (nullable = true)
 |-- home_score: long (nullable = true)
 |-- home_xg: double (nullable = true)
 |-- home_penalty: long (nullable = true)
 |-- away_score: long (nullable = true)
 |-- away_xg: double (nullable = true)
 |-- away_penalty: long (nullable = true)
 |-- home_manager: string (nullable = true)
 |-- home_captain: string (nullable = true)
 |-- away_manager: string (nullable = true)
 |-- away_captain: string (nullable = true)
 |-- home_goals: string (nullable = true)
 |-- away_goals: string (nullable = true)
 |-- Attendance: long (nullable = true)
 |-- Venue: string (nullable = true)
 |-- Officials: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Referee: string (nullable = true)
 |-- Notes: string (nullable = true)
 |-- Round: string (nullable = true)
 |-- Host: string (nullable = true)
 |-- Year: long (nullable = true)



In [2]:
#convert StringType to DateType
matches = matches.withColumn("Date", to_date(col("Date"), 'yyyy-MM-dd'))
matches = matches.withColumn("Year", to_date(col("Year"), 'yyyy'))

matches.printSchema()
matches.show(10)

root
 |-- home_team: string (nullable = true)
 |-- away_team: string (nullable = true)
 |-- home_score: long (nullable = true)
 |-- home_xg: double (nullable = true)
 |-- home_penalty: long (nullable = true)
 |-- away_score: long (nullable = true)
 |-- away_xg: double (nullable = true)
 |-- away_penalty: long (nullable = true)
 |-- home_manager: string (nullable = true)
 |-- home_captain: string (nullable = true)
 |-- away_manager: string (nullable = true)
 |-- away_captain: string (nullable = true)
 |-- home_goals: string (nullable = true)
 |-- away_goals: string (nullable = true)
 |-- Attendance: long (nullable = true)
 |-- Venue: string (nullable = true)
 |-- Officials: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Score: string (nullable = true)
 |-- Referee: string (nullable = true)
 |-- Notes: string (nullable = true)
 |-- Round: string (nullable = true)
 |-- Host: string (nullable = true)
 |-- Year: date (nullable = true)

+---------+-----------+----------+----

Calculate the total number and average number of goals per year

In [27]:
matches_all_goals = matches.withColumn("total_goals", col("home_score") + col("away_score")) \
    .groupBy('Year', 'Host') \
    .agg(sum('total_goals').alias('total_goals')) \
    .sort('Year') \
    .na.drop(how = 'any') \

matches_all_goals.show(21)

+----------+--------------------+-----------+
|      Year|                Host|total_goals|
+----------+--------------------+-----------+
|1930-01-01|             Uruguay|         70|
|1934-01-01|               Italy|         70|
|1938-01-01|              France|         84|
|1950-01-01|              Brazil|         88|
|1954-01-01|         Switzerland|        140|
|1958-01-01|              Sweden|        126|
|1962-01-01|               Chile|         89|
|1966-01-01|             England|         89|
|1970-01-01|              Mexico|         95|
|1974-01-01|             Germany|         97|
|1978-01-01|           Argentina|        102|
|1982-01-01|               Spain|        146|
|1986-01-01|              Mexico|        132|
|1990-01-01|               Italy|        115|
|1994-01-01|       United States|        141|
|1998-01-01|              France|        171|
|2002-01-01|Korea Republic, J...|        161|
|2006-01-01|             Germany|        147|
|2010-01-01|        South Africa| 

In [28]:
from pyspark.sql.functions import *

matches_avg_goals = matches.withColumn("total_goals", col("home_score") + col("away_score")) \
    .groupBy('Year', 'Host') \
    .agg(avg('total_goals').alias('average_goals_per_match')) \
    .sort('Year') \
    .na.drop(how = 'any') \


matches_avg_goals.show(21)

+----------+--------------------+-----------------------+
|      Year|                Host|average_goals_per_match|
+----------+--------------------+-----------------------+
|1930-01-01|             Uruguay|      3.888888888888889|
|1934-01-01|               Italy|      4.117647058823529|
|1938-01-01|              France|      4.666666666666667|
|1950-01-01|              Brazil|                    4.0|
|1954-01-01|         Switzerland|      5.384615384615385|
|1958-01-01|              Sweden|                    3.6|
|1962-01-01|               Chile|                2.78125|
|1966-01-01|             England|                2.78125|
|1970-01-01|              Mexico|                2.96875|
|1974-01-01|             Germany|     2.5526315789473686|
|1978-01-01|           Argentina|     2.6842105263157894|
|1982-01-01|               Spain|     2.8076923076923075|
|1986-01-01|              Mexico|     2.5384615384615383|
|1990-01-01|               Italy|     2.2115384615384617|
|1994-01-01|  

In [31]:
#join the two tables together into one
joinExpression = ["Year", "Host"]
merged_goals = matches_all_goals.join(matches_avg_goals, joinExpression, "left")
merged_goals.show(21)

+----------+--------------------+-----------+-----------------------+
|      Year|                Host|total_goals|average_goals_per_match|
+----------+--------------------+-----------+-----------------------+
|1930-01-01|             Uruguay|         70|      3.888888888888889|
|1934-01-01|               Italy|         70|      4.117647058823529|
|1938-01-01|              France|         84|      4.666666666666667|
|1950-01-01|              Brazil|         88|                    4.0|
|1954-01-01|         Switzerland|        140|      5.384615384615385|
|1958-01-01|              Sweden|        126|                    3.6|
|1962-01-01|               Chile|         89|                2.78125|
|1966-01-01|             England|         89|                2.78125|
|1970-01-01|              Mexico|         95|                2.96875|
|1974-01-01|             Germany|         97|     2.5526315789473686|
|1978-01-01|           Argentina|        102|     2.6842105263157894|
|1982-01-01|        

Save data into bucket/big query

In [23]:
#write data to bucket
matches.write.mode("overwrite").format("csv").save("gs://de_jads_batch_data/matches.csv") # bucket name !

In [32]:
#use the cloud storage bucket for temporary BigQuery export data used by the connector
bucket = "de_jads_temp_annelies" # bucket name !
spark.conf.set('temporaryGcsBucket', bucket)

#save the combined matches data to BigQuery -> do not forget to change project ID
matches.write.format('bigquery') \
    .option('table', 'de2022-362620.assignment2dataset.matches') \
    .mode("overwrite") \
    .save()

#save the goal data to bigQuery -> do not forget to change project ID
merged_goals.write.format('bigquery') \
    .option('table', 'de2022-362620.assignment2dataset.goals') \
    .mode("append") \
    .save()

In [33]:
# stop the spark context
spark.stop()